<a href="https://colab.research.google.com/github/zhenghaojiang/rl_dsge/blob/main/PF_Toy_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [111]:
import numpy as np
import math
import matplotlib.pyplot as plt
from numpy.random import normal,uniform
from scipy.stats import t,gamma,norm

In [112]:

np.random.seed(1111)

params_init = {'alpha': 0.15,
               'beta': 0.7,
               'sigma': 0.7}

params_real = {'alpha': 0.3,
               'beta': 0.5,
               'sigma': 0.5}

def xtrans(x,w,alpha):
    return 0.5+alpha*x/(1+x**2)+w

def sim_y(params: dict = params_real, T = 500) -> np.array:
    alpha = params['alpha']
    beta = params['beta']
    sigma = params['sigma']

    x = 1.0
    y = []
    for _ in range(T):
        x_new = xtrans(x,normal(0,sigma),alpha)
        y_new = x_new + t.rvs(2,1)
        y = np.append(y,y_new)
        x = x_new
    return y


def prior(params: dict):
    alpha = params['alpha']
    beta = params['beta']
    sigma = params['sigma']

    p_alpha = gamma.pdf(alpha,2,scale=0.15)
    p_beta = gamma.pdf(beta,2,scale=0.5)
    p_sigma = gamma.pdf(sigma,2,scale=0.5)
    return p_alpha * p_beta * p_sigma

def loglikelihoodfunc(params: dict, N = 1000, T = 500):
    alpha = params['alpha']
    beta = params['beta']
    sigma = params['sigma']

    y_real = sim_y()
    loglikeli = 0.0
    x = np.ones(N)
    for i in range(T):
        w = normal(0,sigma,N)
        xcond = xtrans(x,w,alpha)
        y = y_real[i] # In practice, change to real y
        v = y - beta * xcond
        q = t.pdf(v,df=2)/sum(t.pdf(v,df=2))
        x = np.random.choice(xcond,N,replace=True,p=q)
        loglikeli = loglikeli + math.log(np.mean(t.pdf(v,df=2)))
    return loglikeli

def mcmc(M = 50, params_init: dict = params_init):

    def threshold(params_new: dict, params_old: dict):
        prior_mol = prior(params_new)
        loglikeli_mol = loglikelihoodfunc(params_new)
        prior_den = prior(params_old)
        loglikeli_den = loglikelihoodfunc(params_old)
        log_res = math.log(prior_mol) + loglikeli_mol - math.log(prior_den) - loglikeli_den
        return math.exp(log_res)
    
    params = params_init
    # params_vec = []
    for _ in range(M):
        params_p = {'alpha': max(params['alpha'] + normal(scale=0.03),0.01),
                    'beta': max(params['beta'] + normal(scale=0.05),0.01),
                    'sigma': max(params['sigma'] + normal(scale=0.05),0.01)}
        chi = uniform()
        if chi <= threshold(params_p,params):
            params_up = params_p
        else:
            params_up = params
        params = params_up
        print(params)

In [110]:
mcmc()

{'alpha': 0.15, 'beta': 0.7, 'sigma': 0.7}
{'alpha': 0.15, 'beta': 0.7, 'sigma': 0.7}
{'alpha': 0.20793904065608687, 'beta': 0.741064821591342, 'sigma': 0.7042614635453676}
{'alpha': 0.20793904065608687, 'beta': 0.741064821591342, 'sigma': 0.7042614635453676}
{'alpha': 0.20793904065608687, 'beta': 0.741064821591342, 'sigma': 0.7042614635453676}
{'alpha': 0.2394564856400182, 'beta': 0.6807865527569101, 'sigma': 0.7859888599773134}
{'alpha': 0.2202119018392754, 'beta': 0.6568696969293784, 'sigma': 0.83704698954056}
{'alpha': 0.18316191516348657, 'beta': 0.6001938981276389, 'sigma': 0.8873876673566035}
{'alpha': 0.12611054657571716, 'beta': 0.6222269484804636, 'sigma': 0.907735565803306}
{'alpha': 0.12611054657571716, 'beta': 0.6222269484804636, 'sigma': 0.907735565803306}
{'alpha': 0.12640251931828056, 'beta': 0.6609407887534067, 'sigma': 0.9811495580930639}
{'alpha': 0.10999695776206794, 'beta': 0.5358551652418301, 'sigma': 0.9853873905280234}
{'alpha': 0.09843263503740252, 'beta': 0.49